# Cross Validation & Ensemble
cross vaildation

kfold_sets = [(tr1, val1), (tr2, val2), ...]    
-> for tr, val in kfold_sets:    
    model.fil(data)   
    model.save_pretrain(모델이름)

ensemble

models = [1, 2, 3]     
for model in models:    
    위에 있는 코드

In [3]:
# hugging face transformers 라이브러리 설치
!pip3 install torch torchvision torchaudio

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ----- ---------------------------------- 0.2/1.2 MB 4.6 MB/s eta 0:00:01
     ------------ --------------------------- 0.4/1.2 MB 4.6 MB/s eta 0:00:01
     ---------------------- ----------------- 0.7/1.2 MB 6.1 MB/s eta 0:00:01
     ----------------------- ---------------- 0.7/1.2 MB 4.5 MB/s eta 0:00:01
     ---------------------------- ----------- 0.8/1.2 MB 4.4 MB/s eta 0:00:01
     ------------------------------------ --- 1.1/1.2 MB 5.0 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ---- ----------------------------------- 0.2/2.1 MB 6.7 MB/s eta 0:00:01
     ---------- ----------------------------- 0.6/2.1 MB 8.8 MB/s eta 0:00:01
     -------------- ------------------------- 0.7/2.1 MB 7.8 MB/s eta 0:00:01
     ------------------ --------------------- 1.0/2.1 MB 6.8 MB/s eta 0:00:01


In [2]:
!pip install keras==2.4.3
!pip install tensorflow==2.2

  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.4.3 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tensorflow==2.2 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.14.0rc0, 2.14.0rc1)
ERROR: No matching distribution found for tensorflow==2.2


In [5]:
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re

from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from transformers import TFBertForSequenceClassification

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
No module named 'tensorflow.keras.layers'

In [2]:

from transformers import AutoTokenizer, TFAutoModel

In [6]:
model = TFAutoModel.from_pretrained("seoyeon96/KcELECTRA-MLM", from_pt=False)
tokenizer = AutoTokenizer.from_pretrained("seoyeon96/KcELECTRA-MLM")

ValueError: Unrecognized model identifier in seoyeon96/KcELECTRA-MLM. Should contains one of 'bert', 'openai-gpt', 'gpt2', 'transfo-xl', 'xlnet', 'xlm', 'roberta'

## Configuration

In [2]:
num_labels = 2
n_splits = 5
batch_size = 64
num_train_epochs = 10

## 데이터 불러오기

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [4]:
display(train.head())
display(test.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [5]:
# 중복제거
train.drop_duplicates(subset=['document'], inplace=True)

In [6]:
# 데이터 분포 확인
train.label.value_counts()

0    73342
1    72841
Name: label, dtype: int64

In [7]:
# 결측치 확인
train.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [8]:
# 결측치 제거
train.dropna(inplace=True)

In [9]:
# 인코딩 전 test data의 결측치 제거
test = test.dropna()
print(len(test))

49997


In [10]:
# 데이터셋 크기 줄이기
num_train = 10000
num_test = 5000

train = train[:num_train]
test = test[:num_test]

In [11]:
train.shape, test.shape

((10000, 3), (5000, 3))

### Tokenizer를 이용한 인코딩

In [13]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("klue/bert-base")

OSError: Model name 'klue/bert-base' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased). We assumed 'klue/bert-base' was a path or url to a configuration file named config.json or a directory containing such a file but couldn't find any such file at this path or url.

In [ ]:
from transformers import BertTokenizerFast, BertModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

## Cross Validation

In [ ]:
X, y = train['document'], train['label']
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=100)
splitgen = skf.split(X, y) # splitgen : index 뭉치 리턴

KFold에 의한 5번의 모델 학습 결과를 각각 따로 저장해보세요.

In [ ]:
os.mkdir('./saved')

In [ ]:
idx = 1
for train_index, val_index in splitgen:

    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    X_train_list = X_train.tolist()
    X_val_list = X_val.tolist()
    y_train = y_train.tolist()
    y_val = y_val.tolist()

    X_train = tokenizer(X_train_list, truncation=True, padding=True, return_tensors = 'tf')
    X_val = tokenizer(X_val_list, truncation=True, padding=True, return_tensors = 'tf')

    train_dataset = tf.data.Dataset.from_tensor_slices((
        dict(X_train),
        y_train
    ))

    val_dataset = tf.data.Dataset.from_tensor_slices((
        dict(X_val),
        y_val
    ))

    model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2, from_pt=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    callback_earlystop = EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.001,
        patience=2)

    model.fit(
        train_dataset.shuffle(num_train).batch(32), epochs=1, batch_size=32,
        validation_data = val_dataset.shuffle(num_test).batch(32),
        callbacks = [callback_earlystop]
    )

    ## model 저장하기
    tokenizer.save_pretrained('./saved/klue_bert_base_tokenizer' + str(idx))
    model.save_pretrained()
    idx += 1


----------------------------------------------------------------------------


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


250/250 [==============================] - 95s 193ms/step - loss: 0.3848 - accuracy: 0.8281 - val_loss: 0.2984 - val_accuracy: 0.8740



----------------------------------------------------------------------------


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


250/250 [==============================] - 94s 191ms/step - loss: 0.3762 - accuracy: 0.8311 - val_loss: 0.3044 - val_accuracy: 0.8695



----------------------------------------------------------------------------


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


250/250 [==============================] - 94s 191ms/step - loss: 0.3717 - accuracy: 0.8364 - val_loss: 0.3341 - val_accuracy: 0.8580


## Ensemble
- 아래 세 종류의 모델에 대해 각 KFold(k=5)를 수행하여 학습시켜, 각 학습결과를 saved 폴더에 저장해보세요.

In [ ]:
models = ['klue/roberta-base', 'bert-base-multilingual-uncased', 'xlm-roberta-base']


- 학습시킨 모델을 각각 불러와서 학습 결과를 합쳐서 앙상블해보세요.


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('saved/' + saved_model)
tokenizer = AutoTokenizer.from_pretrained('saved/' + saved_model)

In [ ]:
X, y = test['document'], test['label']

X_test_list = X.tolist()
y_test = y.tolist()

X_test = tokenizer(X_test_list, truncation=True, padding=True, return_tensors = 'tf')

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test),
    y_test
))

In [ ]:
pred_logits = model.predict(test_dataset).logits
pred = pred_logits.argmax(axis=1)

In [ ]:
# accuracy 계산
acc = np.mean(tf.cast(tf.math.equal(target, pred), tf.float32))